In [ ]:
!pip install panel
!pip install ipywidgets
!pip install plotly
!pip install pandas

In [ ]:
import pandas as pd
import plotly.express as px
import panel as pn
pn.extension()
from ipywidgets import interactive
import ipywidgets as widgets

In [ ]:
url = 'https://raw.githubusercontent.com/JustynaWitulska/Test-repo/main/results_robust_tunned.csv'
df = pd.read_csv(url)

df['N'] = df['n1'] + df['n2']
df['change point'] = df['n1']/df['N']

# Sample size widget
N_list = list(df['N'].unique())
N_dropdown = widgets.Dropdown(
    options=N_list,
    value=N_list[0],
    description='Sample size: ',
    disabled=False)

# Changepoint widget
Changepoint_list = list(df['change point'].unique())
Changepoint_dropdown = widgets.Dropdown(
    options=Changepoint_list,
    value=Changepoint_list[0],
    description='Change point: ',
    disabled=False)

# Changepoint widget
Alpha_list = list(df['alpha'].unique())
Alpha_dropdown = widgets.Dropdown(
    options=Alpha_list,
    value=Alpha_list[0],
    description='alpha: ',
    disabled=False)

# Output
report_output = widgets.Output()
display(report_output)

# Update figures
def filter_function(N, alpha, changepoint):
    df_plot = df[(df.alpha==alpha) & (df['change point']==changepoint) & (df.N==N)]
    df_plot = pd.melt(df_plot, id_vars=['gamma2'], value_vars=['OLS original', 'OLS robust', 'OLS robust tunned',
                                                               'ICSS original', 'ICSS robust', 'ICSS robust tunned'])
    df_plot.loc[:,'gamma2'] = df_plot['gamma2'].apply(lambda x: str(x))

    with report_output:
        report_output.clear_output()

        fig = px.box(df_plot, x='gamma2', y='value', color='variable')
        fig.add_hline(y=int(N*changepoint), line_color="black", opacity=0.5) #line_dash="dash",
        fig.update_layout(legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ))
        fig.show()


interactive(filter_function, N=N_list, alpha=Alpha_list, changepoint=Changepoint_list)